In [2]:
import tensorflow as tf
import pandas as pd
import re
from langdetect import detect
from transformers import pipeline

2025-04-23 15:39:31.523220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745415572.460904   93980 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745415572.759566   93980 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745415575.273617   93980 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745415575.273645   93980 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745415575.273647   93980 computation_placer.cc:177] computation placer alr

In [4]:
#load the dataset
df = pd.read_csv("reviews.csv")

# Clean html tags
def clean_html(text):
    if isinstance(text, str):
        return re.sub(r"<.*?>", "", text)
    return text


# Remove the empty comments rows
df = df.dropna(subset=["comments"])

df["comments"] = df["comments"].apply(clean_html)

In [5]:
# Filter out the rows that isn't in English
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False
    
df["is_english"] = df["comments"].apply(is_english)
df = df[df["is_english"]]



In [7]:
print(df["comments"])

0         great fun at lidia's. she had the power adapte...
1         Great centrally located room, very nice facili...
2         Lidia is a very nice person. Very good plase t...
3         Great host, although we were late in our arriv...
4         Good location (for us), the bed was good and t...
                                ...                        
144934    Everything was good. The owner was really frie...
144935    Oskar was great with his communication and alw...
144938    Erica’s apartment was perfect for us. It has e...
144942    A really lovely apartment with lots of charact...
144944    Had a great stay at Jairo’s flat! The room its...
Name: comments, Length: 99702, dtype: object


In [ ]:
# load huggingface sentiment pipline
sentiment_pipeline = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9937177896499634}]


In [6]:
# apply sentiment analysis
def get_sentiment(text):
    try:
        result = sentiment_pipeline(text[:512])[0]
        return result["label"]
    except:
        return "UNKNOWN"
    
df["sentiment"] = df["comments"].apply(get_sentiment)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
# Save the results
df.to_csv("airbnb_reviews_with_sentiment.csv", index=False)


In [10]:
print(df[["comments", "sentiment"]])

                                                 comments sentiment
0       great fun at lidia's. she had the power adapte...  POSITIVE
1       Great centrally located room, very nice facili...  POSITIVE
2       Lidia is a very nice person. Very good plase t...  POSITIVE
3       Great host, although we were late in our arriv...  POSITIVE
4       Good location (for us), the bed was good and t...  POSITIVE
...                                                   ...       ...
144934  Everything was good. The owner was really frie...  POSITIVE
144935  Oskar was great with his communication and alw...  POSITIVE
144938  Erica’s apartment was perfect for us. It has e...  POSITIVE
144942  A really lovely apartment with lots of charact...  POSITIVE
144944  Had a great stay at Jairo’s flat! The room its...  POSITIVE

[99702 rows x 2 columns]
